In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import *
import string
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import warnings
from random import choice
from gensim import corpora, models
from gensim.models import CoherenceModel
import csv
from liwc import LIWC
warnings.filterwarnings("ignore")

# Load Raw Data

In [16]:
df=pd.read_csv("../data/labeled_data.csv")
df.drop(df.columns[[0]], axis=1,inplace=True)
print(df.iloc[:5,:])
print(df.shape)

   count  hate_speech  offensive_language  neither  class  \
0      3            0                   0        3      2   
1      3            0                   3        0      1   
2      3            0                   3        0      1   
3      3            0                   2        1      1   
4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
(24783, 6)


# Data Pre-processing 

In [17]:
df['tweet'][:20]

0     !!! RT @mayasolovely: As a woman you shouldn't...
1     !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2     !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3     !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4     !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5     !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6     !!!!!!"@__BrighterDays: I can not just sit up ...
7     !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8     " &amp; you might not get ya bitch back &amp; ...
9     " @rhythmixx_ :hobbies include: fighting Maria...
10    " Keeks is a bitch she curves everyone " lol I...
11                   " Murda Gang bitch its Gang Land "
12    " So hoes that smoke are losers ? " yea ... go...
13        " bad bitches is the only thing that i like "
14                              " bitch get up off me "
15                      " bitch nigga miss me with it "
16                               " bitch plz whatever "
17                            " bitch who do you

In [18]:
# Index of tweet only contain format like"#XX #XX" (which would be empty after remove hashtag)
# We need to specially deal with them
special_index=[804,826,846,848,849,923,1016,1122,1909,3398,4818,5711,6098,6279,6332,6668,7168,11951,15859,18062]
# for example
print(df['tweet'][804])

#Yankees


In [19]:
# Also there are two meaningless tweet
print(df['tweet'][4828])
print(df['tweet'][6098])
meaningless_index=[4828,6098]

@TNKidsFoodPorn @Oreo ..............&#128530;
@hoes &#9829;


In [20]:
#copy original text
txt=df.iloc[:,4:6]
tweet=txt.iloc[:,1].copy(deep=True)
stemmer = PorterStemmer()

In [21]:
def preprocess(tweet_,specail_index,meaningless_index):
    #remove stopwords
    stopwords = nltk.corpus.stopwords.words("english")
    other = ["#ff", "ff", "rt"]
    stopwords.extend(other)
    #word tokenize and remove stopwords
    
    for i in range(tweet_.shape[0]):
        text=tweet_[i].lower()
        text1=''.join([word+" " for word in text.split() if word not in stopwords])
        tweet_[i]=text1
    
    #remove digit and excessive whitespace
    #remove url mention and hashtag
    
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    
    for i in range(tweet_.shape[0]):
        text_string=tweet_[i]
        parsed_text = re.sub(space_pattern, ' ', text_string)
        parsed_text = re.sub(giant_url_regex, '', parsed_text)
        parsed_text = re.sub(mention_regex, '', parsed_text) 
        if i not in specail_index:
            parsed_text = re.sub(hashtag_regex, '', parsed_text)
        else:
            parsed_text = re.sub('#','',parsed_text)
        if i in meaningless_index:
            parsed_text ='This is a simple tweet'
        tweet_[i]=parsed_text
    
    #tokenize and stemming
    """Removes punctuation and stems tweets. Returns a list of stemmed tokens."""
    for i in range(tweet_.shape[0]):
        
        tweet_stem=tweet_[i]
        
        remove = str.maketrans('','',string.punctuation) 
        tweet_stem = tweet_stem.translate(remove).strip()

        tweet_process = [stemmer.stem(t) for t in tweet_stem.split()]
        
        #tweet_[i]=tweet_process
        
        tweet_[i] = ' '.join([str(elem) for elem in tweet_process]) 
        

        
    return tweet_

## Results of pre-processing:

In [22]:
#results of pre-process
tweet_=preprocess(tweet,special_index,meaningless_index)
print(tweet[:20])

0     woman complain clean hous amp man alway take t...
1     boy dat coldtyga dwn bad cuffin dat hoe 1st place
2            dawg ever fuck bitch start cri confus shit
3                                      look like tranni
4        shit hear might true might faker bitch told ya
5       shit blow meclaim faith somebodi still fuck hoe
6              sit hate anoth bitch got much shit go on
7            caus im tire big bitch come us skinni girl
8                  amp might get ya bitch back amp that
9                       hobbi includ fight mariam bitch
10    keek bitch curv everyon lol walk convers like ...
11                           murda gang bitch gang land
12                            hoe smoke loser yea go ig
13                                 bad bitch thing like
14                                            bitch get
15                                     bitch nigga miss
16                                     bitch plz whatev
17                                           bit

In [23]:
tweet = pd.DataFrame(tweet)
tweet.to_csv("../data/clean.csv", index = False)

# Feature Extraction
## 1.LIWC

In [ ]:
#run LIWC on the whole df

LIWC_list=[]
for i in range(len(X_train)):
    L = LIWC()
    length, matched, labels = L.classify(X_train[i])
    LIWC_list.append(L.readable(labels))
LIWC_list
'''
#update preprocessed text to original df
df.tweet=tweet
hateSpeech=df[df['hate_speech'] ==3]["tweet"].reset_index(drop=True)
#it is the list of hate speech
LIWC_list1=[]
for i in range(len(hateSpeech)):
    L = LIWC()
    length, matched, labels = L.classify(hateSpeech[i])
    LIWC_list1.append(L.readable(labels))
LIWC_list1
#function that add dictionary
def mergeDict(dict1, dict2):
   # Merge dictionaries and keep values of common keys in list
   dict3 = {**dict1, **dict2}
   for key, value in dict3.items():
       if key in dict1 and key in dict2:
               dict3[key] = value +dict1[key]
   return dict3
#add those dic together
hate_dic=LIWC_list1[0]
for i in range(len(LIWC_list1)):
    hate_dic=mergeDict(hate_dic, LIWC_list1[i])
hate_dic.keys()
#find hate_relevent key by analyzing hate speech
relevent_key=[k for k, v in hate_dic.items() if v > 50]
'''
#build dataframe with relevent columns
LIWC_df=pd.DataFrame(data=LIWC_list, index=None, columns=None, dtype=None, copy=False)[relevent_key]
LIWC_df.shape

In [ ]:
LIWC_list=[]
for i in range(len(tweet)):
    L = LIWC()
    length, matched, labels = L.classify(tweet[i])
    #I divide the number by length to get fraction of such words in a sentence
    LIWC_list.append(L.readable(labels/len(tweet[i])))

#LIWC_df=pd.DataFrame(data=LIWC_list, index=None, columns=None, dtype=None, copy=False)
#LIWC_df = LIWC_df.fillna(0)
#LIWC_df.to_csv("LIWC.csv", index = False)

In [ ]:
LIWC_df = LIWC_df.fillna(0)
LIWC_df.to_csv("../data/LIWC.csv", index = False)

## Split training and testing

In [24]:
#combine tweet data with labels
tweet_pd=pd.DataFrame(tweet)
tweet_pd['label']=txt.iloc[:,0]
tweet_train, tweet_test = train_test_split(tweet_pd, random_state=17, test_size = 0.1)
print("size of train data/test data:",len(tweet_train),len(tweet_test))

size of train data/test data: 22304 2479


In [25]:
tweet_train['label'].to_csv("../data/y_train.csv", index = False)
tweet_test['label'].to_csv("../data/y_test.csv", index = False)

In [26]:
X_train = tweet_train['tweet']
X_test = tweet_test['tweet']
X_train.to_csv("../data/X_train.csv", index = False)
X_test.to_csv("../data/X_test.csv", index = False)

## 2.Skip-gram


In [27]:
#input data for word2vec
tweet_skip_train=np.array(tweet_train.iloc[:,0])
tweet_skip_train[:3]

array(['frozen pea ibuprofen new best friend crippl',
       'day 2014 yank advanc despit 01 loss', 'im go show bitch done'],
      dtype=object)

word2vec by skip-gram

In [28]:
model = Word2Vec(size=100,min_count=1, workers=5,sg=1)  #dimension is 200, consider 5 words, using skip-gram
model.build_vocab(tweet_skip_train)
model.train(tweet_skip_train,total_examples = model.corpus_count,epochs = 50)

(7974372, 45307100)

In [30]:
#warnings.filterwarnings("ignore")
word_vec=[]
v_in_dic=[]
#average each word vector to get the vector of a single tweet
for i in range(len(tweet_skip_train)):
    sum1=0
    for w in tweet[i]:
        try:
            sum1+=model[w]
            v_in_dic.append(w)
        #for out of dictionary vocab use random select in vocabulary word method to generate vectors
        except:
            random_w=choice(v_in_dic)
            sum1+=model[random_w]
            continue
    avg=sum1/(len(tweet_skip_train))
    avg=list(avg)
    word_vec.append(avg)
#word_vec[:5]     

KeyError: 0

In [33]:
len(word_vec)

22304

Organize the format of the output vector

In [34]:
word_vec_array=np.array(word_vec)
word_vec_l=np.reshape(word_vec_array[0],(1,100))
for i in range(len(tweet_skip_train)):
    array=np.reshape(word_vec_array[i],(1,100))
    word_vec_l=np.r_[word_vec_l,array]
#print(word_vec_l)
word_vec_lt=word_vec_l
word_vec_lt=np.delete(word_vec_lt,0,0)
print(word_vec_lt.shape)

[[ 8.5338004e-05  6.9035600e-06 -2.0924020e-04 ... -1.2735344e-04
  -2.1649488e-04  1.2031146e-04]
 [ 8.5338004e-05  6.9035600e-06 -2.0924020e-04 ... -1.2735344e-04
  -2.1649488e-04  1.2031146e-04]
 [-5.9363474e-06  1.3560270e-05 -1.9345969e-04 ... -1.1438209e-04
  -1.2622922e-04  1.1307213e-04]
 ...
 [ 5.8622296e-05  6.4456355e-05 -2.3561601e-04 ... -1.1445833e-04
  -1.3924936e-04  1.2791694e-04]
 [ 1.3101550e-05  2.2595434e-05 -6.4107742e-05 ... -2.1456435e-05
  -2.4978521e-05  2.6791484e-05]
 [ 3.6837540e-05  1.1829634e-04 -4.5385252e-04 ... -2.3355952e-04
  -2.0691885e-04  2.3315978e-04]]
(22304, 100)


In [35]:
word_vec_lt_ = pd.DataFrame(word_vec_lt)
word_vec_lt_.to_csv("../data/train_ski.csv", index = False)

Construct test vector

In [36]:
tweet_skip_test=np.array(tweet_test.iloc[:,0])
#average each word vector to get the vector of a single tweet
for i in range(len(tweet_test)):
    sum1=0
    for w in tweet[i]:
        try:
            sum1+=model[w]
            v_in_dic.append(w)
        #for out of dictionary vocab use random select in vocabulary word method to generate vectors
        except:
            random_w=choice(v_in_dic)
            sum1+=model[random_w]
            continue
    avg=sum1/(len(tweet_skip_train))
    avg=list(avg)
    word_vec.append(avg)

In [37]:
word_vec_array=np.array(word_vec)
word_vec_l=np.reshape(word_vec_array[0],(1,100))
for i in range(len(tweet_skip_test)):
    array=np.reshape(word_vec_array[i],(1,100))
    word_vec_l=np.r_[word_vec_l,array]
#print(word_vec_l)
word_vec_lt=word_vec_l
word_vec_lt=np.delete(word_vec_lt,0,0)
print(word_vec_lt.shape)

[[ 8.5338004e-05  6.9035600e-06 -2.0924020e-04 ... -1.2735344e-04
  -2.1649488e-04  1.2031146e-04]
 [ 8.5338004e-05  6.9035600e-06 -2.0924020e-04 ... -1.2735344e-04
  -2.1649488e-04  1.2031146e-04]
 [-5.9363474e-06  1.3560270e-05 -1.9345969e-04 ... -1.1438209e-04
  -1.2622922e-04  1.1307213e-04]
 ...
 [-9.5239329e-06  9.1593240e-05 -3.1487900e-04 ... -2.3024407e-04
  -2.6830297e-04  1.9702339e-04]
 [ 2.3259687e-05  9.7768589e-06 -1.3366160e-04 ... -8.4095700e-05
  -8.0500751e-05  8.7899869e-05]
 [ 1.4624413e-05  1.3782654e-05 -3.8958213e-05 ... -7.3622091e-06
   5.6125318e-06  2.0170601e-06]]
(2479, 100)


In [38]:
word_vec_lt_ = pd.DataFrame(word_vec_lt)
word_vec_lt_.to_csv("../data/test_ski.csv", index = False)

In [13]:
#save this array as csv

f = open('../data/word2vec_skip.csv', 'w')
a = csv.writer(f)
a.writerows(word_vec_lt)
f.close()

In [14]:
np.savetxt('../data/word2vec_skip1.csv', word_vec_lt, delimiter = ',')  

## 3.LDA

construct dic for LDA 

In [14]:
tweet = np.array(tweet_train.iloc[:,0])
dictionary = corpora.Dictionary(tweet)
corpus = [dictionary.doc2bow(words) for words in tweet]

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

Find the optimal topic numbers

In [15]:
for i in [50, 60, 70, 80]:
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=i)
    coherence_model_lda = CoherenceModel(model=lda, texts=tweet, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    


Coherence Score:  0.39856848738008344

Coherence Score:  0.4063903418840271

Coherence Score:  0.40174023517653634

Coherence Score:  0.40982003924347604


Coherence Score is always below 0.55, so it might not be a good feature extraction method for this data.

In [16]:
# lda model, num_topics is the number of topic
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=70)
# print 5 words for each topic 
for topic in lda.print_topics(num_words=5):
    print(topic)
# topic infer
lda_infer= lda.inference(corpus)
print(lda.inference(corpus))

(33, '0.253*"yo" + 0.097*"bitch" + 0.085*"ass" + 0.044*"smell" + 0.044*"like"')
(27, '0.198*"littl" + 0.168*"feel" + 0.100*"bitch" + 0.029*"ass" + 0.023*"like"')
(2, '0.233*"nigga" + 0.139*"hoe" + 0.073*"niggah" + 0.067*"n" + 0.056*"bitch"')
(42, '0.121*"bitch" + 0.112*"kill" + 0.089*"way" + 0.069*"pleas" + 0.062*"alreadi"')
(48, '0.203*"love" + 0.134*"u" + 0.091*"bitch" + 0.068*"cuz" + 0.057*"hoe"')
(56, '0.133*"tell" + 0.100*"turn" + 0.093*"boy" + 0.064*"bitch" + 0.044*"dyke"')
(29, '0.105*"watch" + 0.097*"ask" + 0.097*"made" + 0.085*"could" + 0.072*"bitch"')
(63, '0.198*"would" + 0.068*"thought" + 0.059*"cracker" + 0.038*"kick" + 0.037*"redneck"')
(54, '0.093*"peopl" + 0.084*"guy" + 0.078*"say" + 0.052*"bitch" + 0.044*"im"')
(45, '0.154*"me" + 0.129*"bitch" + 0.104*"like" + 0.083*"start" + 0.059*"act"')
(53, '0.154*"even" + 0.095*"kid" + 0.093*"he" + 0.048*"happi" + 0.042*"word"')
(57, '0.090*"last" + 0.089*"night" + 0.075*"care" + 0.058*"him" + 0.047*"whole"')
(18, '0.212*"think" +

In [103]:
print(lda_infer[0].shape)

(24783, 70)


In [104]:
lda_infer = pd.DataFrame(lda_infer[0])
lda_infer.to_csv("../data/lda_infer.csv", index = False)

## 4. TFIDF


In [ ]:
#X = pd.read_csv('../data/clean.csv')['tweet']
X_train=pd.read_csv("../data/X_train.csv")['tweet']
X_test=pd.read_csv("../data/X_test.csv")['tweet']

from sklearn.feature_extraction.text import CountVectorizer
text = X_train.values
text_ = X_test.values
vectorizer = CountVectorizer(ngram_range = (1, 2))
vectorizer.fit(text)
tfidf_train = vectorizer.transform(text)
tfidf_test =vectorizer.transform(text_)
tfidf_train = pd.DataFrame.sparse.from_spmatrix(tfidf_train)
tfidf_test = pd.DataFrame.sparse.from_spmatrix(tfidf_test)
#tfidf_train = pd.DataFrame(tfidf_train)
#tfidf_train.shape

#tfidf_test = pd.DataFrame(tfidf_test)
tfidf_train.to_csv("../data/tfidf_train.csv", index = False)
tfidf_test.to_csv("../data/tfidf_test.csv", index = False)